In [27]:
from sqlalchemy import create_engine
import pandas.io.sql as psql

engine = create_engine('postgresql+psycopg2://w9i@localhost/w9i')

def read_sql_query(query):
    with engine.connect() as connection:
        result = psql.read_sql(query, connection).style.set_properties(**{'text-align': 'left'})
        return result
    
read_sql_query('select * from book')

,book_id,title,author,price,amount
0,1,Мастер и Маргарита,Булгаков М.А.,670.990000,3
1,2,Белая гвардия,Булгаков М.А.,540.500000,5
2,3,Идиот,Достоевский Ф.М.,460.000000,10
3,4,Братья Карамазовы,Достоевский Ф.М.,799.010000,2


# Задание
### В конце года цену всех книг на складе пересчитывают – снижают ее на 30%. Написать SQL запрос, который из таблицы book выбирает названия, авторов, количества и вычисляет новые цены книг. Столбец с новой ценой назвать new_price, цену округлить до 2-х знаков после запятой.

In [7]:
read_sql_query("""
SELECT title, author, amount,
    ROUND(price - (0.3 * price), 2) AS new_price
FROM book
""")

,title,author,amount,new_price
0,Мастер и Маргарита,Булгаков М.А.,3,469.690000
1,Белая гвардия,Булгаков М.А.,5,378.350000
2,Идиот,Достоевский Ф.М.,10,322.000000
3,Братья Карамазовы,Достоевский Ф.М.,2,559.310000


<h2>Выборка данных, вычисляемые столбцы, логические&nbsp;функции</h2>

<p>В SQL реализована возможность заносить в поле значение в зависимости от условия. 
    Для этого используется функция <code>IF()</code>:</p>

<code class="language-sql hljs">IF(логическое_выражение, выражение_1, выражение_2)</code>

<p>Функция вычисляет <code>логическое_выражение,</code> если оно истина – в поле заносится значение <code>выражения_1</code>, в противном случае –&nbsp; значение <code>выражения_2. </code>Все три параметра <code>IF() </code>являются обязательными.</p>

<p>Допускается использование вложенных функций, вместо <code>выражения_1</code> или <code>выражения_2</code> может стоять новая функция <code><strong>IF</strong></code>.</p>

<p><strong>Пример&nbsp;</strong></p>

<p>Для каждой книги из таблицы <code><strong>book</strong></code>&nbsp;установим скидку следующим образом: если количество книг меньше 4, то скидка будет составлять 50% от цены, в противном случае 30%.</p>

<p>В PostgreSQL вместо&nbsp;<code>IF</code>&nbsp; используют конструкцию&nbsp;<code>CASE ... WHEN ... THEN ... ELSE ... END</code>&nbsp;и <a href="https://www.postgresql.org/docs/12/functions-conditional.html" rel="noopener noreferrer nofollow" target="_blank">некоторые другие</a>.</p>

<code class="hljs ada">...
    <span class="hljs-keyword">CASE</span>
        <span class="hljs-keyword">WHEN</span> author = <span class="hljs-symbol">'Text</span> <span class="hljs-number">1</span>' <span class="hljs-keyword">THEN</span> price * <span class="hljs-number">2</span>
        <span class="hljs-keyword">WHEN</span> author = <span class="hljs-symbol">'Text</span> <span class="hljs-number">2</span>' <span class="hljs-keyword">THEN</span> price * <span class="hljs-number">3</span>
        <span class="hljs-keyword">ELSE</span> price <span class="hljs-keyword">END</span>
...</code>

In [9]:
read_sql_query("""
SELECT title, amount, price, 
    ROUND(CASE
        WHEN amount < 4 THEN price*0.5
        ELSE price*0.7 END, 2) AS sale
FROM book
""")

,title,amount,price,sale
0,Мастер и Маргарита,3,670.990000,335.500000
1,Белая гвардия,5,540.500000,378.350000
2,Идиот,10,460.000000,322.000000
3,Братья Карамазовы,2,799.010000,399.510000


<p><strong>Пример</strong></p>

<p>Усложним вычисление скидки в зависимости от количества книг.&nbsp;Если количество книг меньше 4 – то скидка 50%, меньше 11&nbsp;– 30%, в остальных случаях – 10%. И еще укажем какая именно скидка на каждую книгу.</p>

In [30]:
read_sql_query("""
SELECT title, amount, price,
ROUND(
    CASE
        WHEN amount < 4 THEN price * 0.5
        ELSE
            CASE
                WHEN amount < 11 THEN price * 0.7
                ELSE price * 0.9
                END
        END
,2) AS sale,
CASE
    WHEN amount<4 THEN 'скидка 50%%'
    ELSE
        CASE
            WHEN amount < 11 THEN 'скидка 30%%'
            ELSE 'скидка 10%%'
            END
    END AS Ваша_скидка
FROM book
""")

,title,amount,price,sale,Ваша_скидка
0,Мастер и Маргарита,3,670.990000,335.500000,скидка 50%
1,Белая гвардия,5,540.500000,378.350000,скидка 30%
2,Идиот,10,460.000000,322.000000,скидка 30%
3,Братья Карамазовы,2,799.010000,399.510000,скидка 50%
4,Стихотворения и поэмы,15,650.000000,585.000000,скидка 10%


## Задание

#### <p>При анализе продаж книг выяснилось, что наибольшей популярностью пользуются книги Михаила Булгакова, на втором месте книги Сергея Есенина. Исходя из этого решили поднять цену книг Булгакова на 10%, а цену книг Есенина - на 5%. Написать запрос, куда включить автора, название книги и новую цену, последний столбец назвать <code><strong>new_price</strong></code>. Значение округлить до двух знаков после запятой.</p>

In [37]:
read_sql_query("""
SELECT author, title,
ROUND(
CASE
    WHEN author = 'Булгаков М.А.' THEN (price*0.10) + price
    WHEN author = 'Есенин С.А.' THEN (price*0.05) + price
    ELSE price
    END ,2) AS new_price
FROM book
""")

,author,title,new_price
0,Булгаков М.А.,Мастер и Маргарита,738.090000
1,Булгаков М.А.,Белая гвардия,594.550000
2,Достоевский Ф.М.,Идиот,460.000000
3,Достоевский Ф.М.,Братья Карамазовы,799.010000
4,Есенин С.А.,Стихотворения и поэмы,682.500000
